In [1]:
import pandas as pd
import numpy as np
import re
import string

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import spacy

In [2]:
df = pd.read_csv('../../../data/datamarket_productos_de_supermercados.csv')
df.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://tienda.mercadona.es/product/46019/polv...,mercadona-es,cuidado_facial_y_corporal_desodorante,Polvos desodorantes para pies Deliplus,Bote,2.00,2.67,100g,2021-03-06 00:00:00,59d0ad23d8c38cc8f0afb6ffb053233e
1,https://tienda.mercadona.es/product/34180/oreg...,mercadona-es,aceite_especias_y_salsas_especias,Orégano Hacendado,Bote,0.56,3.11,100g,2021-03-06 00:00:00,504fdd80e8835de5e2dc658cbf433588
2,https://tienda.mercadona.es/product/34179/pere...,mercadona-es,aceite_especias_y_salsas_especias,Perejil Hacendado,Bote,0.69,3.83,100g,2021-03-06 00:00:00,0d71f44bf49fc08a48139249b79c68f4
3,https://tienda.mercadona.es/product/19431/hoja...,mercadona-es,aceite_especias_y_salsas_especias,Hoja de laurel Hacendado,Paquete,0.66,5.50,100g,2021-03-06 00:00:00,1e4f48630b17e19d798bafcff5b260e9
4,https://tienda.mercadona.es/product/34131/hier...,mercadona-es,aceite_especias_y_salsas_especias,Hierbas provenzales Hacendado,Bote,0.70,2.80,100g,2021-03-06 00:00:00,3631e71df0d89ff6158343d2778075fa


In [3]:
df.info()

# Analizar si 'insert_date' hay que cambiarlo a formato fecha

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8637232 entries, 0 to 8637231
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      object 
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(2), object(8)
memory usage: 659.0+ MB


# Modificando formato de columna

In [4]:
# Creamos nueva columna 'Date' para que adquiera el formato Datetime

df["Date"] =  pd.to_datetime(df["insert_date"], format="%Y/%m/%d")
df.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Date
0,https://tienda.mercadona.es/product/46019/polv...,mercadona-es,cuidado_facial_y_corporal_desodorante,Polvos desodorantes para pies Deliplus,Bote,2.00,2.67,100g,2021-03-06 00:00:00,59d0ad23d8c38cc8f0afb6ffb053233e,2021-03-06
1,https://tienda.mercadona.es/product/34180/oreg...,mercadona-es,aceite_especias_y_salsas_especias,Orégano Hacendado,Bote,0.56,3.11,100g,2021-03-06 00:00:00,504fdd80e8835de5e2dc658cbf433588,2021-03-06
2,https://tienda.mercadona.es/product/34179/pere...,mercadona-es,aceite_especias_y_salsas_especias,Perejil Hacendado,Bote,0.69,3.83,100g,2021-03-06 00:00:00,0d71f44bf49fc08a48139249b79c68f4,2021-03-06
3,https://tienda.mercadona.es/product/19431/hoja...,mercadona-es,aceite_especias_y_salsas_especias,Hoja de laurel Hacendado,Paquete,0.66,5.50,100g,2021-03-06 00:00:00,1e4f48630b17e19d798bafcff5b260e9,2021-03-06
4,https://tienda.mercadona.es/product/34131/hier...,mercadona-es,aceite_especias_y_salsas_especias,Hierbas provenzales Hacendado,Bote,0.70,2.80,100g,2021-03-06 00:00:00,3631e71df0d89ff6158343d2778075fa,2021-03-06


In [5]:
df.drop('insert_date', axis=1, inplace=True)

In [6]:
df.sort_values(by='Date')
df

,url,supermarket,category,name,description,price,reference_price,reference_unit,product_id,Date
0,https://tienda.mercadona.es/product/46019/polv...,mercadona-es,cuidado_facial_y_corporal_desodorante,Polvos desodorantes para pies Deliplus,Bote,2.00,2.670,100g,59d0ad23d8c38cc8f0afb6ffb053233e,2021-03-06
1,https://tienda.mercadona.es/product/34180/oreg...,mercadona-es,aceite_especias_y_salsas_especias,Orégano Hacendado,Bote,0.56,3.110,100g,504fdd80e8835de5e2dc658cbf433588,2021-03-06
2,https://tienda.mercadona.es/product/34179/pere...,mercadona-es,aceite_especias_y_salsas_especias,Perejil Hacendado,Bote,0.69,3.830,100g,0d71f44bf49fc08a48139249b79c68f4,2021-03-06
3,https://tienda.mercadona.es/product/19431/hoja...,mercadona-es,aceite_especias_y_salsas_especias,Hoja de laurel Hacendado,Paquete,0.66,5.500,100g,1e4f48630b17e19d798bafcff5b260e9,2021-03-06
4,https://tienda.mercadona.es/product/34131/hier...,mercadona-es,aceite_especias_y_salsas_especias,Hierbas provenzales Hacendado,Bote,0.70,2.800,100g,3631e71df0d89ff6158343d2778075fa,2021-03-06
...,...,...,...,...,...,...,...,...,...,...
8637227,https://tienda.mercadona.es/product/58306/codi...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Codillo de jamón entero Incarlopsa,Pieza,1.88,2.090,kg,820da5814b4e22857f6768b51f49fc9d,2022-10-16
8637228,https://tienda.mercadona.es/product/59269/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de Teruel El Ontanar lonchas,Paquete,3.10,25.834,kg,e0bef39645a35d2109bae6b9786110f9,2022-10-16
8637229,https://tienda.mercadona.es/product/59153/punt...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Puntas de jamón curado Incarlopsa,Paquete,1.63,6.500,kg,5b9face859fcedc38f70bffdb18e5482,2022-10-16
8637230,https://tienda.mercadona.es/product/58293/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón serrano Incarlopsa,Pieza,54.00,7.106,kg,b3a69b46a494e9e51264ca335e1a5aa8,2022-10-16


# Buscando df de productos únicos

In [7]:
df.nunique()

# llamativo que el número de product_id sea diferente a 'name'. En este caso, al ser menor y no haber nulos se podría 
# interpretar que hay varios productos con el mismo ID, ya sea xq el nombre del producto las compañías lo modifican por
# temas comerciales pero mantienen el ID.

url                51655
supermarket            3
category            1503
name               39621
description           41
price               2899
reference_price     7221
reference_unit         9
product_id         34547
Date                 590
dtype: int64

In [8]:
# Analizando si el id de un producto se pudiera encontrar en más de 1 supermercado.

print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].name.unique())
print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].name.nunique())
print(df[df['product_id']=='34d9c8bf7db90e988219cef30b3cf02a'].supermarket.unique())

['EL GAITERO sidra extra botella 75 cl']
1
['dia-es']


In [9]:
df_mercadona = df[df['supermarket']=='mercadona-es']
df_dia = df[df['supermarket']=='dia-es']
df_carrefour = df[df['supermarket']=='carrefour-es']

In [10]:
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3493735 entries, 0 to 8637231
Data columns (total 10 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      object        
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   product_id       object        
 9   Date             datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 293.2+ MB


In [11]:
df_mercadona.drop_duplicates(subset='product_id', keep="last", inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\1594156920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mercadona.drop_duplicates(subset='product_id', keep="last", inplace=True)


In [12]:
df_mercadona['product_id'].nunique()

8912

In [13]:
df_dia.drop_duplicates(subset='product_id', keep="last", inplace=True)
df_carrefour.drop_duplicates(subset='product_id', keep="last", inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\476760580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dia.drop_duplicates(subset='product_id', keep="last", inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\476760580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_carrefour.drop_duplicates(subset='product_id', keep="last", inplace=True)


In [14]:
# Entre Carrefour y Dia no coincide ninguno

df_carrefour.merge(df_dia, on='product_id')

,url_x,supermarket_x,category_x,name_x,description_x,price_x,reference_price_x,reference_unit_x,product_id,Date_x,url_y,supermarket_y,category_y,name_y,description_y,price_y,reference_price_y,reference_unit_y,Date_y


In [15]:
# Entre Carrefour y Mercadona no coincide ninguno

df_carrefour.merge(df_mercadona, on='product_id')

,url_x,supermarket_x,category_x,name_x,description_x,price_x,reference_price_x,reference_unit_x,product_id,Date_x,url_y,supermarket_y,category_y,name_y,description_y,price_y,reference_price_y,reference_unit_y,Date_y


In [16]:
# Entre Dia y Mercadona no coincide ninguno

df_dia.merge(df_mercadona, on='product_id')

,url_x,supermarket_x,category_x,name_x,description_x,price_x,reference_price_x,reference_unit_x,product_id,Date_x,url_y,supermarket_y,category_y,name_y,description_y,price_y,reference_price_y,reference_unit_y,Date_y


### Eliminamos duplicados del df original

In [17]:
# Ahora que tenemos la certeza lo que hacemos es quitar todos los id duplicados
df.drop_duplicates(subset='product_id', keep="last", inplace=True)

In [18]:
df

,url,supermarket,category,name,description,price,reference_price,reference_unit,product_id,Date
546,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_acces...,BETER cepillo térmico de cerámica 43 mm 1 ud,NaN,6.75,6.750,ud,2f1fbef47b5cdaf4a5cb4ee65366dee6,2021-03-06
2339,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_laca_...,BONTE agua de peinado cabellos rizados spray 1...,NaN,1.59,10.600,l,12c0fa46042dcd306b3e1de562c457ea,2021-03-06
3233,https://www.dia.es/compra-online/productos/beb...,dia-es,bebidas_isotonicas_y_energeticas_isotonicas,POWERADE bebida refrescante aromatizada ice st...,NaN,1.39,2.780,l,79d9fe5ead25b3f5a8629a67af0f379e,2021-03-06
3954,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_higiene_bucal_dentifricos,SIGNAL pasta dentífrica anticaries tubo 2 x 10...,NaN,2.89,14.450,l,1eedf388186951f78a89dd6e07e17745,2021-03-06
4281,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,COLON detergente máquina líquido flor botella ...,NaN,5.99,0.190,lavado,cd8a878448ae2292c0513aad07abcd37,2021-03-06
...,...,...,...,...,...,...,...,...,...,...
8637227,https://tienda.mercadona.es/product/58306/codi...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Codillo de jamón entero Incarlopsa,Pieza,1.88,2.090,kg,820da5814b4e22857f6768b51f49fc9d,2022-10-16
8637228,https://tienda.mercadona.es/product/59269/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de Teruel El Ontanar lonchas,Paquete,3.10,25.834,kg,e0bef39645a35d2109bae6b9786110f9,2022-10-16
8637229,https://tienda.mercadona.es/product/59153/punt...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Puntas de jamón curado Incarlopsa,Paquete,1.63,6.500,kg,5b9face859fcedc38f70bffdb18e5482,2022-10-16
8637230,https://tienda.mercadona.es/product/58293/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón serrano Incarlopsa,Pieza,54.00,7.106,kg,b3a69b46a494e9e51264ca335e1a5aa8,2022-10-16


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34547 entries, 546 to 8637231
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34547 non-null  object        
 1   supermarket      34547 non-null  object        
 2   category         34547 non-null  object        
 3   name             34533 non-null  object        
 4   description      7669 non-null   object        
 5   price            34547 non-null  float64       
 6   reference_price  34547 non-null  float64       
 7   reference_unit   34524 non-null  object        
 8   product_id       34547 non-null  object        
 9   Date             34547 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 2.9+ MB


In [20]:
df.nunique()

url                34547
supermarket            3
category            1039
name               33007
description           41
price               1663
reference_price     4997
reference_unit         9
product_id         34547
Date                 590
dtype: int64

# Gestión de Nulos

In [21]:
df.isnull().sum()

url                    0
supermarket            0
category               0
name                  14
description        26878
price                  0
reference_price        0
reference_unit        23
product_id             0
Date                   0
dtype: int64

In [22]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

name                 14
description       26878
reference_unit       23
dtype: int64

In [23]:
null_cols = null_cols[null_cols > 0] / len(df) * 100
null_cols

# El más representativo es la descripción que faltan más del 63% pero no es un campo relevante para nuestro análisis
# Por otro lado están aquellos productos que sin tener nombre o precio, sí que nos podrían sobrar aunque representan un %
# muy bajo de todo el df

name               0.040525
description       77.801256
reference_unit     0.066576
dtype: float64

### Analizando 'Description' 

In [24]:
print('Tipo de valores únicos:', df['description'].unique())
print('Nº de valores únicos:', df['description'].nunique() )

Tipo de valores únicos: [nan 'Caja' 'Tubo' 'Paquete' 'Bote' 'Pieza' 'Botella' 'Bandeja' 'Pack-6'
 'Lata' 'Brick' 'Pack-12' 'Tarro' 'Spray' 'Garrafa' 'Frasco' 'Pack-2'
 'Botellín' 'Tarrina' 'Malla' 'Pack-4' 'Tableta' 'Vaso' 'Pack-3' 'Pack-8'
 'Pack-10' 'Granel' 'Bol' 'Sobre' 'Benjamín' 'Pack-5' 'Tarrito' 'Saco'
 '1/2 Pieza' 'Cubo' 'Bolsa' 'Manojo' 'Pack-9' 'Barril' '1/4 Pieza'
 'Pastilla' 'Pack-16']
Nº de valores únicos: 41


In [25]:
# Podríamos concluir que aunque el número de valores únicos no es excesivamente alto (41), sí es cierto que siguen faltando
# más del 63% de descripciones en el dataset y condicionar posibles búsquedas a este campo tal vez ahora no renta.

df.drop('description',axis=1, inplace=True)
df.head()

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date
546,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_acces...,BETER cepillo térmico de cerámica 43 mm 1 ud,6.75,6.75,ud,2f1fbef47b5cdaf4a5cb4ee65366dee6,2021-03-06
2339,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_laca_...,BONTE agua de peinado cabellos rizados spray 1...,1.59,10.60,l,12c0fa46042dcd306b3e1de562c457ea,2021-03-06
3233,https://www.dia.es/compra-online/productos/beb...,dia-es,bebidas_isotonicas_y_energeticas_isotonicas,POWERADE bebida refrescante aromatizada ice st...,1.39,2.78,l,79d9fe5ead25b3f5a8629a67af0f379e,2021-03-06
3954,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_higiene_bucal_dentifricos,SIGNAL pasta dentífrica anticaries tubo 2 x 10...,2.89,14.45,l,1eedf388186951f78a89dd6e07e17745,2021-03-06
4281,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,COLON detergente máquina líquido flor botella ...,5.99,0.19,lavado,cd8a878448ae2292c0513aad07abcd37,2021-03-06


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34547 entries, 546 to 8637231
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34547 non-null  object        
 1   supermarket      34547 non-null  object        
 2   category         34547 non-null  object        
 3   name             34533 non-null  object        
 4   price            34547 non-null  float64       
 5   reference_price  34547 non-null  float64       
 6   reference_unit   34524 non-null  object        
 7   product_id       34547 non-null  object        
 8   Date             34547 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.6+ MB


### Analizando Nulos de precios

In [27]:
df = df[df['name'].notna()]

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34533 entries, 546 to 8637231
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              34533 non-null  object        
 1   supermarket      34533 non-null  object        
 2   category         34533 non-null  object        
 3   name             34533 non-null  object        
 4   price            34533 non-null  float64       
 5   reference_price  34533 non-null  float64       
 6   reference_unit   34510 non-null  object        
 7   product_id       34533 non-null  object        
 8   Date             34533 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.6+ MB


In [29]:
df.isnull().sum()

url                 0
supermarket         0
category            0
name                0
price               0
reference_price     0
reference_unit     23
product_id          0
Date                0
dtype: int64

# Analizar qué categorías me interesan o hay que quitar

In [30]:
sorted(df['category'].unique().tolist())
#print(df['category'].nunique())

['aceite_especias_y_salsas_aceite_vinagre_y_sal',
 'aceite_especias_y_salsas_especias',
 'aceite_especias_y_salsas_mayonesa_ketchup_y_mostaza',
 'aceite_especias_y_salsas_otras_salsas',
 'agua_y_refrescos_agua',
 'agua_y_refrescos_isotonico_y_energetico',
 'agua_y_refrescos_refresco_de_cola',
 'agua_y_refrescos_refresco_de_naranja_y_de_limon',
 'agua_y_refrescos_refresco_de_te_y_sin_gas',
 'agua_y_refrescos_tonica_y_bitter',
 'al_dia_carne_cerdo',
 'al_dia_carne_mixto',
 'al_dia_carne_pavo',
 'al_dia_carne_pollo',
 'al_dia_carne_vacuno',
 'al_dia_charcuteria_y_quesos_cocidos',
 'al_dia_charcuteria_y_quesos_curados',
 'al_dia_charcuteria_y_quesos_quesos',
 'al_dia_frutas_citricos',
 'al_dia_frutas_frutas_de_temporada',
 'al_dia_frutas_frutas_tropicales',
 'al_dia_frutas_manzanas',
 'al_dia_frutas_peras',
 'al_dia_frutas_platanos',
 'al_dia_pescado_y_marisco_ahumados_y_salazones',
 'al_dia_pescado_y_marisco_marisco',
 'al_dia_pescado_y_marisco_pescado_fresco',
 'al_dia_pescado_y_marisco_

In [31]:
# Empezamos asegurándonos lo q contiene el str 'higie'

df_clean = df[df["category"].str.contains("higie|hogar|limpieza|farmacia|cuidado|bebe|mascot|maquillaj|solidari|perfume") == False]
sorted(df_clean['category'].unique().tolist())
print(df_clean['category'].nunique())
#df_clean['category'].value_counts()

644


In [32]:
df_clean
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19386 entries, 3233 to 8637231
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   url              19386 non-null  object        
 1   supermarket      19386 non-null  object        
 2   category         19386 non-null  object        
 3   name             19386 non-null  object        
 4   price            19386 non-null  float64       
 5   reference_price  19386 non-null  float64       
 6   reference_unit   19383 non-null  object        
 7   product_id       19386 non-null  object        
 8   Date             19386 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 1.5+ MB


In [33]:
sorted(df_clean['category'].unique().tolist())

['aceite_especias_y_salsas_aceite_vinagre_y_sal',
 'aceite_especias_y_salsas_especias',
 'aceite_especias_y_salsas_mayonesa_ketchup_y_mostaza',
 'aceite_especias_y_salsas_otras_salsas',
 'agua_y_refrescos_agua',
 'agua_y_refrescos_isotonico_y_energetico',
 'agua_y_refrescos_refresco_de_cola',
 'agua_y_refrescos_refresco_de_naranja_y_de_limon',
 'agua_y_refrescos_refresco_de_te_y_sin_gas',
 'agua_y_refrescos_tonica_y_bitter',
 'al_dia_carne_cerdo',
 'al_dia_carne_mixto',
 'al_dia_carne_pavo',
 'al_dia_carne_pollo',
 'al_dia_carne_vacuno',
 'al_dia_charcuteria_y_quesos_cocidos',
 'al_dia_charcuteria_y_quesos_curados',
 'al_dia_charcuteria_y_quesos_quesos',
 'al_dia_frutas_citricos',
 'al_dia_frutas_frutas_de_temporada',
 'al_dia_frutas_frutas_tropicales',
 'al_dia_frutas_manzanas',
 'al_dia_frutas_peras',
 'al_dia_frutas_platanos',
 'al_dia_pescado_y_marisco_ahumados_y_salazones',
 'al_dia_pescado_y_marisco_marisco',
 'al_dia_pescado_y_marisco_pescado_fresco',
 'al_dia_pescado_y_marisco_

# Buscando máxima automatización

### Spacy y NLTK

In [34]:
spanishstemmer=SnowballStemmer('spanish')

In [35]:
nlp = spacy.load('es_core_news_sm')
def normalize_lemma(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = "".join(lemmas)
    return lemmas_cadena
word_list = normalize_lemma('Mini cereales rellenos de chocolate y avellana 4 kg')
word_list

'minicerealrellenochocolateavellán'

In [36]:
def normalize_lemma_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = " ".join(lemmas)
    return lemmas_cadena

In [37]:
def normalize_stemm(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    stems = [spanishstemmer.stem(token) for token in lexical_tokens]
    stems_cadena = "".join(stems)
    return stems_cadena
word_stemm = normalize_stemm('Mini cereales rellenos de chocolate y avellana 4 kg')
word_stemm

'minicerealrellenchocolatavellan'

In [38]:
def normalize_stemm_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    stems = [spanishstemmer.stem(token) for token in lexical_tokens]
    stems_cadena = " ".join(stems)
    return stems_cadena

In [39]:
df_clean['name_lemma'] = df_clean['name'].apply(lambda x: normalize_lemma(x))
df_clean['name_lemma_space'] = df_clean['name'].apply(lambda x: normalize_lemma_space(x))
df_clean['name_stemm'] = df_clean['name'].apply(lambda x: normalize_stemm(x))
df_clean['name_stemm_space'] = df_clean['name'].apply(lambda x: normalize_stemm_space(x))
df_clean.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\2060938844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['name_lemma'] = df_clean['name'].apply(lambda x: normalize_lemma(x))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\2060938844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['name_lemma_space'] = df_clean['name'].apply(lambda x: normalize_lemma_space(x))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_27512\2060938844.py:3: SettingWithCopyWarning: 
A value is trying to

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date,name_lemma,name_lemma_space,name_stemm,name_stemm_space
3233,https://www.dia.es/compra-online/productos/beb...,dia-es,bebidas_isotonicas_y_energeticas_isotonicas,POWERADE bebida refrescante aromatizada ice st...,1.39,2.78,l,79d9fe5ead25b3f5a8629a67af0f379e,2021-03-06,poweradirbebidarefrescantearomatizadostormzero...,poweradir bebida refrescante aromatizado storm...,poweradbebrefrescaromatizstormzerbotell,powerad beb refresc aromatiz storm zer botell
4593,https://www.carrefour.es/supermercado/pomelo-c...,carrefour-es,el_mercado_frutas_naranjas_y_otros_citricos,Pomelo Círculo de Calidad 1 kg aprox,1.75,1.75,kg,6d8ad09c426320e638209da067f2e5f2,2021-03-06,pomelocírculocalidadaprox,pomelo círculo calidad aprox,pomelcirculcalidaprox,pomel circul calid aprox
4680,https://tienda.mercadona.es/product/9522/mini-...,mercadona-es,cereales_y_galletas_cereales,Mini cereales rellenos de chocolate y avellana...,1.30,7.43,kg,527837fde906593fa80458fd7a9b546e,2021-03-06,minicerealrellenochocolateavellanahacendado,mini cereal relleno chocolate avellana hacendado,minicerealrellenchocolatavellanhacend,mini cereal rellen chocolat avellan hacend
5854,https://www.carrefour.es/supermercado/baguette...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Baguette precocida Carrefour pack de 2 unidade...,0.75,2.50,kg,a06071d024822fcf8e9d12ff897d7b8b,2021-03-06,baguettirprecocidocarrefourpackunidad,baguettir precocido carrefour pack unidad,baguettprecoccarrefourpackunidad,baguett precoc carrefour pack unidad
5856,https://www.carrefour.es/supermercado/pan-de-m...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Pan de molde sin corteza Natural 100% Bimbo si...,2.30,5.11,kg,7e731ecb83c085e8583683cc228d1423,2021-03-06,moldecortezanaturalbimbolactós,molde corteza natural bimbo lactós,moldcorteznaturalbimblactos,mold cortez natural bimb lactos


In [40]:
df_clean.to_csv('../../../data/datamarket_clean_unique_methods.csv')

# Segmentando por Supermercado

In [41]:
df_mercadona = df_clean[df_clean['supermarket']=='mercadona-es']
df_mercadona

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date,name_lemma,name_lemma_space,name_stemm,name_stemm_space
4680,https://tienda.mercadona.es/product/9522/mini-...,mercadona-es,cereales_y_galletas_cereales,Mini cereales rellenos de chocolate y avellana...,1.30,7.430,kg,527837fde906593fa80458fd7a9b546e,2021-03-06,minicerealrellenochocolateavellanahacendado,mini cereal relleno chocolate avellana hacendado,minicerealrellenchocolatavellanhacend,mini cereal rellen chocolat avellan hacend
41175,https://tienda.mercadona.es/product/5343/legum...,mercadona-es,arroz_legumbres_y_pasta_legumbres,Legumbre grano verde Hacendado,1.48,2.960,kg,f214cf93052e9d366d5e81cf7322968c,2021-03-08,legumbregranoverdehacendado,legumbre grano verde hacendado,legumbrgranverdhacend,legumbr gran verd hacend
41777,https://tienda.mercadona.es/product/17415/sals...,mercadona-es,aceite_especias_y_salsas_otras_salsas,Salsa fresca foie Hacendado,1.70,12.140,kg,65f5eb84b264bab0a8db9c5bcad86b4b,2021-03-08,salsarfrescafoiehacendado,salsar fresca foie hacendado,salsfrescfoihacend,sals fresc foi hacend
61572,https://tienda.mercadona.es/product/70708/lomo...,mercadona-es,charcuteria_y_quesos_embutido_curado,Lomo de bellota ibérico Estrella de Castilla 5...,14.10,30.000,kg,09a6f24599b5cba5b4535793fb87dd2f,2021-03-09,lomobellotaibéricoestrellacastillorazaibérico,lomo bellota ibérico estrella castillo raza ib...,lombellotiberestrellcastillraziber,lom bellot iber estrell castill raz iber
70943,https://tienda.mercadona.es/product/11416/infu...,mercadona-es,cacao_cafe_e_infusiones_te_e_infusiones,Infusión fría Frutos del bosque Hacendado,1.30,3.250,100g,68e33ceff6210e22210b74cbc24ce125,2021-03-09,infusiónfríofrutobosquehacendado,infusión frío fruto bosque hacendado,infusionfrifrutbosquhacend,infusion fri frut bosqu hacend
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8637227,https://tienda.mercadona.es/product/58306/codi...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Codillo de jamón entero Incarlopsa,1.88,2.090,kg,820da5814b4e22857f6768b51f49fc9d,2022-10-16,codillojamónenteroincarlopsa,codillo jamón entero incarlopsa,codilljamonenterincarlops,codill jamon enter incarlops
8637228,https://tienda.mercadona.es/product/59269/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de Teruel El Ontanar lonchas,3.10,25.834,kg,e0bef39645a35d2109bae6b9786110f9,2022-10-16,jamónteruelontanarloncha,jamón teruel ontanar loncha,jamonteruelontanlonch,jamon teruel ontan lonch
8637229,https://tienda.mercadona.es/product/59153/punt...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Puntas de jamón curado Incarlopsa,1.63,6.500,kg,5b9face859fcedc38f70bffdb18e5482,2022-10-16,puntajamóncuradoincarlopsa,punta jamón curado incarlopsa,puntjamoncurincarlops,punt jamon cur incarlops
8637230,https://tienda.mercadona.es/product/58293/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón serrano Incarlopsa,54.00,7.106,kg,b3a69b46a494e9e51264ca335e1a5aa8,2022-10-16,jamónserranoincarlopsa,jamón serrano incarlopsa,jamonserranincarlops,jamon serran incarlops


In [42]:
df_carrefour = df_clean[df_clean['supermarket']=='carrefour-es']
df_carrefour

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date,name_lemma,name_lemma_space,name_stemm,name_stemm_space
4593,https://www.carrefour.es/supermercado/pomelo-c...,carrefour-es,el_mercado_frutas_naranjas_y_otros_citricos,Pomelo Círculo de Calidad 1 kg aprox,1.75,1.75,kg,6d8ad09c426320e638209da067f2e5f2,2021-03-06,pomelocírculocalidadaprox,pomelo círculo calidad aprox,pomelcirculcalidaprox,pomel circul calid aprox
5854,https://www.carrefour.es/supermercado/baguette...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Baguette precocida Carrefour pack de 2 unidade...,0.75,2.50,kg,a06071d024822fcf8e9d12ff897d7b8b,2021-03-06,baguettirprecocidocarrefourpackunidad,baguettir precocido carrefour pack unidad,baguettprecoccarrefourpackunidad,baguett precoc carrefour pack unidad
5856,https://www.carrefour.es/supermercado/pan-de-m...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Pan de molde sin corteza Natural 100% Bimbo si...,2.30,5.11,kg,7e731ecb83c085e8583683cc228d1423,2021-03-06,moldecortezanaturalbimbolactós,molde corteza natural bimbo lactós,moldcorteznaturalbimblactos,mold cortez natural bimb lactos
8345,https://www.carrefour.es/supermercado/mostaza-...,carrefour-es,la_despensa_alimentacion_salsas_y_tomate_frito,Mostaza Heinz envase 220 ml.,1.50,6.25,l,89bd2d3e0ba18cd9b0f142b4d196055f,2021-03-06,mostazaheinzenvar,mostaza heinz envar,mostazheinzenvas,mostaz heinz envas
8346,https://www.carrefour.es/supermercado/mayonesa...,carrefour-es,la_despensa_alimentacion_salsas_y_tomate_frito,Mayonesa Ybarra sin gluten y sin lactosa envas...,1.35,3.37,l,a8e9ed74b0fd6a489d278d41efe3e9eb,2021-03-06,mayonésybarraglutarlactósenvase,mayonés ybarra glutar lactós envase,mayonesybarrglutlactosenvas,mayones ybarr glut lactos envas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8636761,https://www.carrefour.es/supermercado/longaniz...,carrefour-es,productos_frescos_carniceria_embutidos_frescos,Longaniza blanca de cerdo fresca ElPozo 650 g,2.99,4.60,kg,c3f48af67b90e95e972a86645e41f2c5,2022-10-16,longanizarblancocerdofrescaelpozo,longanizar blanco cerdo fresca elpozo,longanizblanccerdfrescelpoz,longaniz blanc cerd fresc elpoz
8636762,https://www.carrefour.es/supermercado/chorizo-...,carrefour-es,productos_frescos_carniceria_embutidos_frescos,Chorizo oreado de cerdo Carrefour 400 g,4.50,11.25,kg,a735f1b18a658a6eaeaf7d40ad35dc4a,2022-10-16,choraceroreadocerdocarrefour,choracer oreado cerdo carrefour,chorizorecerdcarrefour,choriz ore cerd carrefour
8636763,https://www.carrefour.es/supermercado/salchich...,carrefour-es,productos_frescos_carniceria_embutidos_frescos,Salchichas de pollo ecológica Carrefour Bio 300 g,6.00,20.00,kg,751a3f86fd35dcedcc8e0efffb3f61f3,2022-10-16,salchichapolloecológicocarrefour,salchicha pollo ecológico carrefour,salchichpollecologcarrefour,salchich poll ecolog carrefour
8636764,https://www.carrefour.es/supermercado/txistorr...,carrefour-es,productos_frescos_carniceria_embutidos_frescos,Txistorra de Navarra De Nuestra Tierra 220 g,2.50,11.36,kg,3f4b02f3feb5efb7591a8362dedf08cf,2022-10-16,txistorranavarratierra,txistorra navarra tierra,txistorrnavarrtierr,txistorr navarr tierr


In [43]:
df_dia = df_clean[df_clean['supermarket']=='dia-es']
df_dia

,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date,name_lemma,name_lemma_space,name_stemm,name_stemm_space
3233,https://www.dia.es/compra-online/productos/beb...,dia-es,bebidas_isotonicas_y_energeticas_isotonicas,POWERADE bebida refrescante aromatizada ice st...,1.39,2.78,l,79d9fe5ead25b3f5a8629a67af0f379e,2021-03-06,poweradirbebidarefrescantearomatizadostormzero...,poweradir bebida refrescante aromatizado storm...,poweradbebrefrescaromatizstormzerbotell,powerad beb refresc aromatiz storm zer botell
13794,https://www.dia.es/compra-online/productos/fre...,dia-es,frescos_carne_y_aves_hamburguesas_y_picadas,ROLER hamburguesas de lomo de vaca bandeja 2 u...,2.95,12.29,kg,d71662b3ba875e702a5d5d3646859e30,2021-03-06,rolerhamburguesalomovacobandeja,roler hamburguesa lomo vaco bandeja,rolhamburgueslomvacbandej,rol hamburgues lom vac bandej
14157,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_curado_fuet_y_longaniza,ARGAL fuet calidad extra envase 120 gr,1.25,11.36,kg,488442529c0030bf98bd77225e357138,2021-03-06,argalfuetcalidadextraenvasir,argal fuet calidad extra envasir,argalfuetcalidextraenvas,argal fuet calid extra envas
14306,https://www.dia.es/compra-online/productos/cha...,dia-es,charcuteria_y_quesos_quesos_tablas,TIGER queso fondue classic caja 400 gr,3.85,9.63,kg,8ae39e814a97b892769f85ad15990bbf,2021-03-06,tigerquesofondueclassiccajo,tiger queso fondue classic cajo,tigquesfonduclassiccaj,tig ques fondu classic caj
27511,https://www.dia.es/compra-online/productos/ali...,dia-es,alimentacion_conservas_vegetales_alcachofa,CELORRIO corazones de alcachofa 8/12 piezas fr...,2.19,13.27,kg,ba656d58dc6908114e4bb11a5a6716ff,2021-03-07,celorriocorazónalcachofapiezafrasco,celorrio corazón alcachofa pieza frasco,celorricorazonalcachofpiezfrasc,celorri corazon alcachof piez frasc
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635125,https://www.dia.es/compra-online/congelados/he...,dia-es,congelados_helados,MAGNUM helado bombón doble chocolate caja 3 ud...,3.89,14.73,kg,0ae0bdf2dfe4791307a0eae834285f3c,2022-10-16,magnumheladobombóndoblechocolatecajar,magnum helado bombón doble chocolate cajar,magnumhelbombondoblchocolatcaj,magnum hel bombon dobl chocolat caj
8635126,https://www.dia.es/compra-online/congelados/he...,dia-es,congelados_helados,MAGNUM helado bombón after dinner classic caja...,4.99,17.21,kg,c7b29ff960925c3c9f2ae11a85016d9b,2022-10-16,magnumheladobombónafterdinnerclassiccajo,magnum helado bombón after dinner classic cajo,magnumhelbombonafterdinnclassiccaj,magnum hel bombon after dinn classic caj
8635127,https://www.dia.es/compra-online/congelados/he...,dia-es,congelados_helados,DIA TEMPTATION tarta de whisky caja 531 gr,2.19,4.12,kg,e488ad2c08411abdf21220279a407b09,2022-10-16,temptationtartawhiskycajo,temptation tarta whisky cajo,temptationtartwhiskycaj,temptation tart whisky caj
8635128,https://www.dia.es/compra-online/congelados/he...,dia-es,congelados_helados,DIA TEMPTATION helado sabor stracciatella barq...,2.45,4.67,kg,17b89c3b574f7dc7ad72e935801f119b,2022-10-16,temptationheladosaborstracciatellobarqueta,temptation helado sabor stracciatello barqueta,temptationhelsaborstracciatellbarquet,temptation hel sabor stracciatell barquet
